In [2]:
from imblearn.combine import SMOTEENN
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score, make_scorer

# 1. ADVANCED RESAMPLING (SMOTE-ENN)
# This cleans the noisy synthetic data points created in Milestone 3
sme = SMOTEENN(random_state=42)
X_train_clean, y_train_clean = sme.fit_resample(X_train, y_train)

# 2. DEFINE F2-SCORER (Recall importance = 2 * Precision importance)
f2_scorer = make_scorer(fbeta_score, beta=2)

# 3. CONSTRUCT STACKING META-ENSEMBLE
# Level-0: Tuned Base Learners
# Level-1: Meta-Learner (Logistic Regression)
stack_clf = StackingClassifier(
    estimators=[
        ('xgb', best_xgb),
        ('rf', best_rf)
    ],
    final_estimator=LogisticRegression(),
    stack_method='predict_proba',
    passthrough=True # Allows Meta-Learner to see raw features too
)

# 4. OPTIMIZED PIPELINE
final_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', stack_clf)
])

final_pipe.fit(X_train_clean, y_train_clean)

# 5. DYNAMIC THRESHOLD OPTIMIZATION (THE "KNEE" METHOD)
y_probs = final_pipe.predict_proba(X_test)[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)

# We find the threshold that yields Recall >= 0.85 with the highest Precision
mask = recalls >= 0.85
idx = np.where(mask)[0][-1]
optimal_threshold = thresholds[idx]

y_pred_final = (y_probs >= optimal_threshold).astype(int)

NameError: name 'best_rf' is not defined